In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
includet("../VARs/utils.jl")
includet("../VARs/VAR_OLS.jl")

In [7]:
df = DataFrame(XLSX.readtable("../../data/usa_macro_data.xlsx", "Quarterly", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :observation_date), Dates.year, first)

72×9 TimeArray{Float64, 2, Date, Matrix{Float64}} 1953-04-01 to 2024-01-01
┌────────────┬─────────────────┬─────────┬───────┬──────┬───────────┬───────────
│            │ A939RX0Q048SBEA │ GDPDEF  │ TB3MS │ GS10 │ Price_10Y │ Stock_Pr ⋯
├────────────┼─────────────────┼─────────┼───────┼──────┼───────────┼───────────
│ 1953-04-01 │         18341.0 │   13.42 │  2.15 │  3.0 │   103.357 │       24 ⋯
│ 1954-01-01 │         17636.0 │  13.544 │  1.06 │ 2.44 │    110.23 │       26 ⋯
│ 1955-01-01 │         18396.0 │  13.675 │  1.22 │ 2.65 │   111.433 │       36 ⋯
│ 1956-01-01 │         18658.0 │  14.104 │  2.33 │  2.9 │   112.895 │       44 ⋯
│ 1957-01-01 │         18881.0 │  14.621 │   3.1 │  3.4 │   112.455 │       43 ⋯
│ 1958-01-01 │         18032.0 │  14.982 │  1.76 │ 3.04 │   117.098 │       41 ⋯
│ 1959-01-01 │         19041.0 │  15.224 │  2.77 │ 3.99 │   112.898 │       54 ⋯
│ 1960-01-01 │         19614.0 │  15.428 │  3.87 │ 4.49 │   111.742 │       55 ⋯
│     ⋮      │        ⋮        │  

In [26]:
freq = 1

cpi = percentchange(data_source[:GDPDEF], :log) .* 100
gdp = percentchange(data_source[:A939RX0Q048SBEA], :log) .* 100
sr = log1p.(data_source[:TB3MS] ./ freq) .- cpi
term = log1p.(data_source[:GS10] ./ freq) .- sr
Δd = round.(percentchange(data_source[:Dividend], :log) .* 100, digits=2) .- cpi

dy = round.((log.(data_source[:Dividend]) .- log.(data_source[:Stock_Price])), digits=2)

data= merge(gdp, cpi, sr, term, Δd, dy)

start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)
s_data = to(from(data, start),end_d)

display(returns_summarystats(s_data, freq)) 
display(cor(values(s_data)))



,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
A939RX0Q048SBEA,2.3223,7.351,1.9276,3.225,-4.6008,-0.4525,0.8822,0.0342,1.0544,0.83,1.8746
GDPDEF,2.1525,10.3724,3.1652,3.9088,0.5552,1.457,1.858,0.8134,1.7222,1.4705,2.3529
TB3MS_GDPDEF,1.8608,0.6921,-1.7233,-0.4616,-8.4628,-1.5252,-2.1623,0.7611,2.2307,-0.9261,-1.2443
GS10_TB3MS_GDPDEF,2.0394,10.6076,3.5137,4.066,1.1934,1.499,2.1137,0.7927,1.9331,1.7229,2.9136
Dividend_GDPDEF,5.6106,14.4243,2.3734,6.2816,-23.0452,-1.1509,-0.9204,0.4748,4.5049,0.423,2.5805
Dividend_Stock_Price,0.4079,-2.84,-3.6203,-3.3325,-4.42,-0.1632,-3.9475,0.8961,-0.9867,-8.876,-3.525


nothing

6×6 Matrix{Float64}:
  1.0        -0.165751   0.240752  -0.203922   0.152106  -0.0936008
 -0.165751    1.0       -0.947618   0.984713  -0.208237   0.508928
  0.240752   -0.947618   1.0       -0.97889    0.199814  -0.394245
 -0.203922    0.984713  -0.97889    1.0       -0.233569   0.481152
  0.152106   -0.208237   0.199814  -0.233569   1.0       -0.11648
 -0.0936008   0.508928  -0.394245   0.481152  -0.11648    1.0

In [27]:
s_data

70×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1955-01-01 to 2024-01-01
┌────────────┬─────────────────┬──────────┬──────────────┬───────────────────┬──
│            │ A939RX0Q048SBEA │ GDPDEF   │ TB3MS_GDPDEF │ GS10_TB3MS_GDPDEF │ ⋯
├────────────┼─────────────────┼──────────┼──────────────┼───────────────────┼──
│ 1955-01-01 │          4.2191 │  0.96257 │    -0.165063 │           1.45979 │ ⋯
│ 1956-01-01 │         1.41418 │  3.08891 │     -1.88594 │           3.24691 │ ⋯
│ 1957-01-01 │         1.18811 │  3.60004 │     -2.18905 │           3.67066 │ ⋯
│ 1958-01-01 │        -4.60082 │  2.43906 │     -1.42383 │           2.82008 │ ⋯
│ 1959-01-01 │         5.44466 │  1.60236 │     -0.27529 │           1.88273 │ ⋯
│ 1960-01-01 │          2.9649 │  1.33109 │     0.252003 │           1.45093 │ ⋯
│ 1961-01-01 │        -2.49858 │  1.10869 │     0.100273 │           1.46626 │ ⋯
│ 1962-01-01 │         5.70945 │  1.31187 │   0.00185362 │           1.61158 │ ⋯
│     ⋮      │        ⋮        │  

In [28]:
assets_names = colnames(s_data)
T, n_assets = size(s_data)
model = VARModel(s_data)
model_summary(model)

,const,A939RX0Q048SBEA,GDPDEF,TB3MS_GDPDEF,GS10_TB3MS_GDPDEF,Dividend_GDPDEF,Dividend_Stock_Price,std
A939RX0Q048SBEA,4.066,0.0299,-1.0047,0.5857,1.3377,-0.0308,0.7398,2.3385
GDPDEF,3.1702,0.2035,0.9304,-0.6935,-0.7631,-0.0341,0.4847,1.1765
TB3MS_GDPDEF,-2.7836,-0.1335,-0.2101,1.4361,0.8512,0.0327,-0.3812,1.1883
GS10_TB3MS_GDPDEF,3.4194,0.1467,0.2704,-0.6661,-0.113,-0.0372,0.4677,1.2025
Dividend_GDPDEF,-12.8402,0.9217,-5.4799,1.8858,7.4932,0.4757,-1.7952,4.2905
Dividend_Stock_Price,-0.6106,0.0252,0.0012,0.0032,0.0301,0.0038,0.8802,0.1452


,A939RX0Q048SBEA,GDPDEF,TB3MS_GDPDEF,GS10_TB3MS_GDPDEF,Dividend_GDPDEF,Dividend_Stock_Price
A939RX0Q048SBEA,1.0,-0.08,0.21,-0.17,0.19,-0.38
GDPDEF,-0.08,1.0,-0.97,0.98,-0.1,0.19
TB3MS_GDPDEF,0.21,-0.97,1.0,-0.99,0.13,-0.24
GS10_TB3MS_GDPDEF,-0.17,0.98,-0.99,1.0,-0.15,0.22
Dividend_GDPDEF,0.19,-0.1,0.13,-0.15,1.0,0.13
Dividend_Stock_Price,-0.38,0.19,-0.24,0.22,0.13,1.0


In [29]:
scenarios = simulate(model, 25*freq, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, 0.025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  

,A939RX0Q048SBEA,GDPDEF,TB3MS_GDPDEF,GS10_TB3MS_GDPDEF,Dividend_GDPDEF,Dividend_Stock_Price
1,1.9428,2.3732,-0.5439,2.1848,1.7768,-4.26
5,1.5213,2.2769,-0.8472,2.4468,0.5786,-4.2417
15,1.5918,2.2377,-1.083,2.6015,2.324,-4.1635
25,1.6058,2.3707,-1.2462,2.7638,2.5455,-4.0984


,A939RX0Q048SBEA,GDPDEF,TB3MS_GDPDEF,GS10_TB3MS_GDPDEF,Dividend_GDPDEF,Dividend_Stock_Price
1,0.0,0.0,0.0,0.0,0.0,0.0
5,0.8869,1.0809,1.0032,1.0526,2.5981,0.1422
15,0.5477,1.2489,1.0358,1.1745,1.8371,0.2402
25,0.4357,1.1527,0.9018,1.0745,1.5054,0.2722


,A939RX0Q048SBEA,GDPDEF,TB3MS_GDPDEF,GS10_TB3MS_GDPDEF,Dividend_GDPDEF,Dividend_Stock_Price
1,1.0,1.0,1.0,1.0,NaN,1.0
5,-0.014,0.0118,-0.0158,0.0108,-0.0136,-0.0266
15,-0.0078,0.0672,-0.0601,0.0697,-0.0072,0.027
25,0.0317,0.0276,-0.0282,0.0296,-0.036,-0.0043


,A939RX0Q048SBEA,GDPDEF,TB3MS_GDPDEF,GS10_TB3MS_GDPDEF,Dividend_GDPDEF,Dividend_Stock_Price
1,-2.0,-2.0,-2.0,-2.0,NaN,-2.0
5,-0.0155,-0.013,-0.0334,-0.0303,0.0434,-0.0676
15,-0.1107,0.0501,0.0355,0.04,0.0309,0.0283
25,-0.0553,0.0315,0.0025,0.0174,-0.028,0.0052


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,1.9428,1.9428,1.9428,1.9428,1.9428,1.9428,1.9428,1.9428,1.9428
5,-0.549,-0.2403,0.0452,0.9303,1.5249,2.1239,2.9748,3.2513,3.6001
15,0.3437,0.5208,0.6899,1.2211,1.5865,1.9683,2.5007,2.6482,2.8466
25,0.6064,0.7709,0.8919,1.3114,1.6042,1.8987,2.329,2.4738,2.6337


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,2.3732,2.3732,2.3732,2.3732,2.3732,2.3732,2.3732,2.3732,2.3732
5,-0.234,0.1584,0.5116,1.5502,2.2637,3.007,4.0556,4.3803,4.7963
15,-0.5624,-0.1615,0.2048,1.3848,2.233,3.0659,4.3223,4.7248,5.2001
25,-0.29,0.1284,0.4972,1.5764,2.3658,3.1571,4.2677,4.5991,5.0563


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.5439,-0.5439,-0.5439,-0.5439,-0.5439,-0.5439,-0.5439,-0.5439,-0.5439
5,-3.1564,-2.8391,-2.5042,-1.5331,-0.8354,-0.1751,0.796,1.1371,1.4945
15,-3.5399,-3.14,-2.7955,-1.7757,-1.0768,-0.3773,0.5981,0.901,1.2686
25,-3.3418,-3.0014,-2.7226,-1.8526,-1.247,-0.6301,0.2328,0.5253,0.8112


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,2.1848,2.1848,2.1848,2.1848,2.1848,2.1848,2.1848,2.1848,2.1848
5,0.0238,0.3816,0.7218,1.7434,2.4274,3.1661,4.1788,4.5229,4.8697
15,-0.0596,0.3583,0.6884,1.8048,2.5927,3.3868,4.556,4.9505,5.3793
25,0.2916,0.6628,1.0209,2.027,2.7589,3.4971,4.5431,4.8489,5.2848


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,1.7768,1.7768,1.7768,1.7768,1.7768,1.7768,1.7768,1.7768,1.7768
5,-5.6005,-4.4859,-3.68,-1.1618,0.5655,2.3279,4.8764,5.6734,6.6269
15,-1.9624,-1.3165,-0.7014,1.0838,2.3513,3.5484,5.3434,5.9285,6.6608
25,-1.0224,-0.4395,0.0668,1.5244,2.5448,3.5726,5.02,5.4693,5.9469


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-4.26,-4.26,-4.26,-4.26,-4.26,-4.26,-4.26,-4.26,-4.26
5,-4.5757,-4.5194,-4.4786,-4.3383,-4.2393,-4.146,-4.0096,-3.966,-3.9146
15,-4.7214,-4.6276,-4.5565,-4.3261,-4.1666,-4.0019,-3.769,-3.6918,-3.601
25,-4.7344,-4.637,-4.5447,-4.2804,-4.0974,-3.9162,-3.6558,-3.5683,-3.4668


In [30]:
display(exp(-4.5497))
display(exp(-4.2882))
display(exp(-4.0979))
display(exp(-3.9079))
display(exp(-3.647))

0.010570375020739566

0.01372961637820637

0.016607514588447565

0.02008263033416715

0.02606921924221897